In [ ]:
import numpy as np
import pandas as pd
from astropy.coordinates import *
import astropy.units as u

import sys,os
sys.path.append(os.path.join(os.getcwd(),".."))
#jupyter的运行路径是文件所在路径，这里要添加上一级的路径到系统变量才能import utils
from utils import ball
from tqdm import tqdm
from astropy.io import fits

In [ ]:
file_path="J_AJ_152_50_table3.dat.gz.fits"
with fits.open(file_path) as hdul:
    hdul.info()
    
    column=hdul[1].data.columns
    table=hdul[1].data

In [ ]:
def coordinate(row):
    # z=d*np.sin(np.deg2rad(la))
    # x=d*np.cos(np.deg2rad(la))*np.cos(np.deg2rad(lo))
    # y=d*np.cos(np.deg2rad(la))*np.sin(np.deg2rad(lo))
    # return d,lo,la,np.array([x,y,z]).reshape(1,3)
    c=SkyCoord(
        sgl = row[1]*u.degree,
        sgb = row[2]*u.degree,
        distance = row[0]*ball.scale_factor(5)*u.mpc,
        frame = "supergalactic"
    )
    return [c.cartesian.x.value,c.cartesian.y.value,c.cartesian.z.value]

coordinate_matrix=np.stack([table["Dist"],table["SGLON"],table["SGLAT"]], axis=1)
coordinate_matrix=np.apply_along_axis(coordinate, 1, coordinate_matrix)
coordinate_matrix=np.array(coordinate_matrix)
print(coordinate_matrix[:5])

In [ ]:
def name(PGC,Abell,GName,Name):
    s="PGC"+str(PGC)+" "
    if Abell:
        s+=str(Abell)+" "
    if GName:
        s+=str(GName)+" "
    if Name:
        s+=str(Name)
    return s.strip()

name_list=[]
for row in tqdm(table):
    name_list.append( name(row["PGC"],row["Abell"],row["GName"],row["Name"]))

print(name_list[:5])

In [ ]:
for i in range(5):
    print(name_list[i],coordinate_matrix[i])

In [ ]:
ball.generateBallObj(coordinate_matrix,os.path.join("./", os.path.basename(file_path).split(".")[0]))

In [ ]:
# looking for Andromeda
for i in range(len(name_list)):
    if "PGC2557" in name_list[i]:
        print(name_list[i])
        print(coordinate_matrix[i])
